In [1]:
! git clone https://github.com/seshuad/IMagenet
! ls 'IMagenet/tiny-imagenet-200/'

Cloning into 'IMagenet'...
remote: Enumerating objects: 120594, done.
remote: Total 120594 (delta 0), reused 0 (delta 0), pack-reused 120594
Receiving objects: 100% (120594/120594), 212.68 MiB | 13.00 MiB/s, done.
Resolving deltas: 100% (1115/1115), done.
Updating files: 100% (120206/120206), done.
test  train  val  wnids.txt  words.txt


In [2]:
import time
import imageio
import numpy as np
from keras.utils import to_categorical

path = 'IMagenet/tiny-imagenet-200/'

def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open(path + 'wnids.txt', 'r')):
        id_dict[line.strip()] = i
    return id_dict

def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open(path + 'words.txt', 'r')):
        n_id, word = line.split('\t')[:2]
        all_classes[n_id] = word.strip()
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])
    return result

def get_data(id_dict):
    print('starting loading data')
    train_data, test_data = [], []
    train_labels, test_labels = [], []
    t = time.time()
    for key, value in id_dict.items():
        for i in range(500):
            img_path = f'{path}train/{key}/images/{key}_{i}.JPEG'
            img = imageio.imread(img_path)
            if img.shape != (64, 64, 3):
                continue
            train_data.append(img)
            train_labels.append(value)

    for line in open(f'{path}val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        img = imageio.imread(f'{path}val/images/{img_name}')
        if img.shape != (64, 64, 3):
            continue
        test_data.append(img)
        test_labels.append(id_dict[class_id])

    print('finished loading data, in {} seconds'.format(time.time() - t))
    return np.array(train_data), to_categorical(train_labels, 200), np.array(test_data), to_categorical(test_labels, 200)


class_to_id = get_class_to_id_dict()

train_data, train_labels, test_data, test_labels = get_data(get_id_dictionary())
print("train data shape: ", train_data.shape)
print("train label shape: ", train_labels.shape)
print("test data shape: ", test_data.shape)
print("test_labels.shape: ", test_labels.shape)


starting loading data


<ipython-input-2-2f3a657d405e>:33: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(img_path)
<ipython-input-2-2f3a657d405e>:41: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(f'{path}val/images/{img_name}')


finished loading data, in 44.478628396987915 seconds
train data shape:  (98179, 64, 64, 3)
train label shape:  (98179, 200)
test data shape:  (9832, 64, 64, 3)
test_labels.shape:  (9832, 200)


In [3]:
def shuffle_data(train_data, train_labels ):
    size = len(train_data)
    train_idx = np.arange(size)
    np.random.shuffle(train_idx)

    return train_data[train_idx], train_labels[train_idx]

train_data, train_labels = shuffle_data(train_data, train_labels)

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, UpSampling2D, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

In [5]:
train_datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
)

In [6]:
train_generator = train_datagen.flow(train_data, train_labels, batch_size=64)
val_datagen = ImageDataGenerator()
validation_generator = val_datagen.flow(test_data, test_labels, batch_size=64)

In [7]:
from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=3,
    verbose=1,
    factor=0.6,
    min_lr=1e-6)

In [8]:
from tensorflow.keras.applications.resnet50 import ResNet50
resnet_model = ResNet50(
    include_top = False,
    weights = 'imagenet',
    input_shape = (224,224,3)
)

for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

94765736/94765736 [==============================] - 5s 0us/step


In [9]:
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [10]:

model = Sequential([
    UpSampling2D(size=(3, 3), interpolation='bilinear'),
    UpSampling2D(size=(2, 2), interpolation='bilinear'),
    ResNet50(include_top=False, weights='imagenet', input_shape=(384, 384, 3)),
    GlobalAveragePooling2D(),
    Dropout(0.25),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dense(200, activation='softmax')
])

In [11]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9)

In [12]:
learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=3,
    verbose=1,
    factor=0.6,
    min_lr=1e-6
)

In [13]:
model.compile(
    optimizer = optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [14]:
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    callbacks=[learning_rate_reduction]
)

print(model.summary())

Epoch 1/30
1535/1535 [==============================] - 436s 267ms/step - loss: 3.6130 - accuracy: 0.2517 - val_loss: 2.4242 - val_accuracy: 0.4541 - lr: 0.0010
Epoch 2/30
1535/1535 [==============================] - 405s 264ms/step - loss: 2.0069 - accuracy: 0.5314 - val_loss: 1.9425 - val_accuracy: 0.5342 - lr: 0.0010
Epoch 3/30
1535/1535 [==============================] - 405s 264ms/step - loss: 1.5208 - accuracy: 0.6304 - val_loss: 1.6322 - val_accuracy: 0.6032 - lr: 0.0010
Epoch 4/30
1535/1535 [==============================] - 405s 264ms/step - loss: 1.2662 - accuracy: 0.6832 - val_loss: 1.4611 - val_accuracy: 0.6446 - lr: 0.0010
Epoch 5/30
1535/1535 [==============================] - 405s 264ms/step - loss: 1.0834 - accuracy: 0.7249 - val_loss: 1.4421 - val_accuracy: 0.6461 - lr: 0.0010
Epoch 6/30
1535/1535 [==============================] - 405s 264ms/step - loss: 0.9534 - accuracy: 0.7537 - val_loss: 1.4281 - val_accuracy: 0.6490 - lr: 0.0010
Epoch 7/30
1535/1535 [============